 # Agrimetrics GraphQL API Functions.

 © 2021 Agrimetrics

 ## Import required libraries
 The GraphQL queries are made with the requests library.

In [1]:
import os
import requests
from functools import lru_cache


 ## General GraphQL API query function

In [2]:
GRAPHQL_ENDPOINT = "https://api.agrimetrics.co.uk/graphql"


@lru_cache
def headers():
    if "API_KEY" in os.environ:
        api_key = os.getenv("API_KEY")
    else:
        api_key = input("Query API Subscription Key: ").strip()

    return {
        "Accept": "application/json",
        "Ocp-Apim-Subscription-Key": api_key,
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip, deflate, br",
    }


def graphql(query, **variables):
    response = requests.post(
        GRAPHQL_ENDPOINT,
        headers=headers(),
        json={"query": query, "variables": variables},
    )
    response.raise_for_status()
    results = response.json()
    errors = results.get("errors")
    if not errors:
        return results["data"]
    raise Exception(f"GraphQL query errors: {errors}")



 ## Field Identifier query

In [3]:
def fields(roi, dataset_id, sowncrop_dataset_id):
    query = """
    query getFieldidsForROI (
        $roi: CoordinateScalar!,
        $dataset_id: [ID!]!,
        $sowncrop_dataset_id: [ID!]!,
        $cursor: String
    ){
        fields(
            geoFilter: {location: {type: Polygon, coordinates: $roi}},
            where: {dataSetId: {EQ: $dataset_id}},
            after: $cursor) {
                id
                cursor
                sownCrop (where: {dataSetId: {EQ: $sowncrop_dataset_id}}){
                    cropType
                    harvestYear
                }
            }
    }"""

    cursor = ""
    while cursor is not None:
        response = graphql(
            query,
            roi=roi,
            dataset_id=dataset_id,
            sowncrop_dataset_id=sowncrop_dataset_id,
            cursor=cursor,
        )
        cursor = response["fields"][-1].pop("cursor", None)
        field_data = response["fields"]

        for field in field_data:
            for crop in field["sownCrop"]:
                yield {
                    "id": field["id"],
                    "year": crop["harvestYear"],
                    "crop": crop["cropType"],
                }



 ## Field weather query

In [4]:
def weather(field_id, start_date, end_date):
    dataset_id = "https://data.agrimetrics.co.uk/data-sets/3987721c-2677-405b-96b1-0d4f2ba721f3"
    query = """
    query getWeatherForField (
        $field_id: [ID!]!,
        $dataset_id: [ID!]!,
        $start_date: Date!,
        $end_date: Date!,
        $cursor: String
    ){
        fields(where: {id: {EQ: $field_id}}) {
            weatherObservations(
                dateRange: {startDate: $start_date, endDate: $end_date},
                after: $cursor
            ){
                solarInsolationDaily      (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                temperatureMaxDaily       (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                temperatureMeanDaily      (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                temperatureMinDaily       (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                windSpeedMeanDaily        (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                rainfallTotalDaily        (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                relativeHumidityMeanDaily (where: {dataSetId: {EQ: $dataset_id}}) { value dateTime }
                cursor
            }
        }
    }"""

    cursor = ""
    while cursor is not None:
        response = graphql(
            query,
            field_id=field_id,
            dataset_id=dataset_id,
            start_date=start_date,
            end_date=end_date,
            cursor=cursor,
        )
        cursor = response["fields"][0]["weatherObservations"].pop("cursor", None)
        for dimension, records in response["fields"][0]["weatherObservations"].items():
            for record in records:
                yield {"fieldId": field_id, "dimension": dimension, **record}



 ## Field centroid query

In [5]:
def centroid(field_id):
    query = """
    query getFieldCentroid ($field_id: [ID!]!){
        fields(where: {id: {EQ: $field_id}}) {
            location {
                centroid
            }
        }
    }"""
    response = graphql(query, field_id=field_id)
    return response["fields"][0]["location"]["centroid"]



 ## Field soil query

In [6]:
def soil(field_id):
    query = """query getSoilForField ($field_id: [ID!]!){
        fields(where: {id: {EQ: $field_id}}) {
            id
            soil {
                topSoil {
                    texture {
                        sandPercentage
                        siltPercentage
                        clayPercentage
                        type
                    }
                    chemicalProperties {
                        carbonConcentration {
                            value
                        }
                    }
                }
            }
        }
    }"""
    response = graphql(query, field_id=field_id)
    return response["fields"][0]["soil"]
